In [ ]:
!pip install ultralytics

In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import numpy as np

In [ ]:
# 製造測試影片

# 原始影片檔案太大，直接跑model.track會讓colab直接讓RAM過載，因此先將影片每五幀截取一張，壓成影片再進行預測
# 有另一種方式是修改model.track裡面的vid_stride，預設1是每幀讀，數字越大跳過幀數越多，例如model.track(test.mp4, vid_stride=5)可以達成一樣每五幀截取一張的效果
# 不過壓成影片會花的時間和運算元多很多，不如直接截取,但取得的是影片檔但RAM不足時推薦使用

In [ ]:
test_path = '/content/drive/MyDrive/MOT17/test/MOT17-03-FRCNN/img1/'
selected_test_path = []
for i in range(100,800,5):# 由100開始是因為圖片檔名為000001，使用1或2位數會造成圖片檔名少了0因此錯誤，如果一定需要可以添加判斷式 if i<10 : print(f'00{i}') elif i<100: print(f'0{i}')來解決
  selected_test_path.append(f'{test_path}000{i}.jpg')
print(selected_test_path[:3])

In [ ]:
# 獲取測試影片大小
sample_img = cv2.imread(selected_test_path[0]) # 取第一張作為樣本就可以知道影片大小
height, width, layers = sample_img.shape

In [ ]:
# 參考資料 官網 https://docs.opencv.org/3.4/dd/d43/tutorial_py_video_display.html

output_video_path = "/content/selected_test.mp4" # 影片輸出位置
   # 寫入格式.mp4使用的是mp4v
video_writer = cv2.VideoWriter(output_video_path, fourcc, 6, (width, height)) # 參數分別為(輸出位置，編碼，每秒幀數，圖片大小) 原片30幀，因裁為1/5，因此參數放入6幀
# 寫入影片
for path in selected_test_path:
    img = cv2.imread(path)
    video_writer.write(img)
# 關閉video_writer
video_writer.release()
cv2.destroyAllWindows()
print(f"影片位置{output_video_path}")

In [ ]:
# 資料測試
# 程式源自yolov8官網教學 https://docs.ultralytics.com/zh/modes/track/#persisting-tracks-loop
# 結合cv2影片存取
model = YOLO('/content/runs/detect/train/weights/best.pt')

# 用cap擷取影片
video_path = output_video_path
cap = cv2.VideoCapture(video_path)

# 儲存track_history 用於ID標註等
track_history = defaultdict(lambda: [])
# 影片存取器
video_writer_result = cv2.VideoWriter('/content/Tracking_result.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 6, (width, height))

while cap.isOpened():
    # 讀取影格
    success, frame = cap.read()

    if success:
        # 跑YOLOv8，參數寫在官網
        results = model.track(frame, persist=True, show_labels=False, show_conf=False, save_txt=True)
        # reset track_id
        track_ids = []

        # 取得Box和track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # 疊BOX及track IDs至圖片
        annotated_frame = results[0].plot()

        # 畫出追蹤

        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # 劃出軌跡

            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(151, 255, 255), thickness=8)

        # 顯示含有Box、track IDs、軌跡的圖，但在colab跑，需要使用cv2_imshow，不能用原本的cv2.imshow()

        video_writer_result.write(annotated_frame)

        # 跑cv2_imshow也會吃運算元，我寫入影片了，就不將結果顯示，如果需要就拿掉註解
        	# 在本地運行使用cv2.imshow()需要多加一個瀏覽器名稱
	        # 例如cv2.imshow(‘annotated_frame’, annotated_frame)
        # cv2_imshow(annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# 最後加入，用於關閉cap和writer
cap.release()
video_writer_result.release()
cv2.destroyAllWindows()